## kerasのh5をpbファイル化

In [3]:
import tensorflow as tf
from tensorflow.contrib.keras.python.keras import backend as K
from tensorflow.contrib.keras.python.keras.models import load_model, Sequential

from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import predict_signature_def

sess = tf.Session()
K.set_session(sess)

# disable loading of learning nodes
K.set_learning_phase(0)



In [4]:
# load model
model = load_model('model.01-0.529-0.752.h5')
config = model.get_config()
weights = model.get_weights()
new_Model = Sequential.from_config(config)
new_Model.set_weights(weights)

# export current best model
export_path = "build/tensorflow_model/"
builder = saved_model_builder.SavedModelBuilder(export_path)

signature = predict_signature_def(inputs={'accelerations': new_Model.input},
                                  outputs={'scores': new_Model.output})





In [5]:
with K.get_session() as sess:
    builder.add_meta_graph_and_variables(sess=sess,
                                         tags=[tag_constants.SERVING],
                                         signature_def_map={
                                             signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature})
    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'build/tensorflow_model/saved_model.pb'
